#### ID 2000

```Write a query that returns binary description of rate type per loan_id. The results should have one row per loan_id and two columns: for fixed and variable type.```

In [ ]:
%%sql
SELECT loan_id,
       CASE WHEN rate_type = 'fixed' THEN 1 ELSE 0 END    AS fixed,
       CASE WHEN rate_type = 'variable' THEN 1 ELSE 0 END AS variable
FROM submissions;

In [ ]:
df = submissions
pd.get_dummies(df[['loan_id','rate_type']], prefix='', prefix_sep='')

#### ID 2002

```Write a query that returns the user ID of all users that have created at least one ‘Refinance’ submission and at least one ‘InSchool’ submission.```

In [ ]:
%%sql
SELECT DISTINCT user_id
FROM loans
WHERE user_id IN (SELECT user_id FROM loans WHERE type IN ('Refinance'))
  AND user_id IN (SELECT user_id FROM loans WHERE type IN ('InSchool'))

In [ ]:
df_ref = loans.query('type == "Refinance"')['user_id'].drop_duplicates()
df_sch = loans.query('type == "InSchool"')['user_id'].drop_duplicates()
df = pd.merge(df_ref, df_sch, on='user_id')

#### ID 2010

```List the top 10 users who accumulated the most sessions where they had more streaming sessions than viewing. Return the user_id, number of streaming sessions, and number of viewing sessions.```

In [ ]:
%%sql
WITH filtered AS (SELECT user_id,
                    COUNT(CASE WHEN session_type = 'streamer' THEN 1 ELSE NULL END) AS streaming_sessions,
                    COUNT(CASE WHEN session_type = 'viewer' THEN 1 ELSE NULL END)   AS viewing_sessions
             FROM twitch_sessions
             GROUP BY user_id
             HAVING COUNT(CASE WHEN session_type = 'streamer' THEN 1 ELSE NULL END) >
                    COUNT(CASE WHEN session_type = 'viewer' THEN 1 ELSE NULL END)),
     ranked AS (SELECT user_id,
                       streaming_sessions,
                       viewing_sessions,
                      DENSE_RANK()
                      OVER (ORDER BY (streaming_sessions + viewing_sessions) DESC) AS rnk
               FROM filtered)
SELECT user_id, streaming_sessions, viewing_sessions
FROM ranked
WHERE rnk <= 10

In [ ]:
# TODO

#### ID 2016

```Which partners have ‘pizza’ in their name and are located in Boston? And what is the average order amount? Output the partner name and the average order amount.```

In [ ]:
%%sql
SELECT pp.name, AVG(po.amount) AS avg
FROM postmates_orders po
         LEFT JOIN postmates_markets pm ON po.city_id = pm.id
         LEFT JOIN postmates_partners pp ON po.seller_id = pp.id
WHERE pm.name = 'Boston'
  AND pp.name ILIKE '%pizza%'
GROUP BY pp.name

In [ ]:
df = pd.merge(pd.merge(postmates_orders, postmates_markets, how='left', left_on='city_id', right_on='id'), postmates_partners, how='left', left_on='seller_id', right_on='id')
df.query('name_x == "Boston" & name_y.str.contains("pizza", case=False)').groupby('name_y', as_index=False).agg(avg=('amount', 'mean'))